## Import Libraries

In [1]:
#import required libraries
import requests
from bs4 import BeautifulSoup
import os

## Get To The Correct Landing Page & Pull Site HTML Content

In [2]:
#cdc_newsroom_url = "https://www.cdc.gov/media/releases/2020/archives.html"

#the specific cdc feed on COVID-19 
cdc_newsroom_url = "https://www.cdc.gov/media/dpk/diseases-and-conditions/coronavirus/coronavirus-2020.html"

#step 2: get to the landing page of the website
landing_page = requests.get(cdc_newsroom_url)

#step 3: convert page into html
soup = BeautifulSoup(landing_page.content, "html.parser")


## Navigate to the Collection of Press Releases

Note: The data pulled from the CDC website comes specifically from the following tabs: PRESS RELEASES, TRANSCRIPTS, + STATEMENTS (and possibly one more tab)


In [11]:
press_release_feed = soup.find_all("div", class_= "pressrel")  #this will include in

## this feed includes the press releases that are in all of the tabs  on the website
## CONTAINS CONTENT FROM THE TAB: PRESS RELEASES, TRANSCRIPTS, + STATEMENTS ON THE CDC'S WEBSITE
#print(press_release_feed)

## Extract Pertinent Transcript Information & Store Results in a Dictionary

In [39]:
#step 1: gets all of the contents of each of the tabs (Advisories, Trasnricripts, etc)

transcript_info = {} #key = air date, value is a dictionary with the (transcript url, guest, and text of the show)
starting_url = "https://www.cdc.gov"

for tab in press_release_feed:
   
    item_titles = tab.find_all("a", class_="item-title")
    pub_dates = tab.find_all("span", class_="item-pubdate")
 
    for (title, date) in zip(item_titles, pub_dates):
        transcript_url = starting_url + title["href"]
        date_text = date.text
        title_text = title.text
        #print(date_text, title_text, transcript_url, "\n")

        #add the information to the transcript info to the dictionary
        transcript_info[date_text] =  {"transcript_url": transcript_url,
                                        "transcript_title": title_text }

#    # pub_dates = tab.find("span", class_="item_pubdate") #.text
  #  print(pub_dates)
   
# <ul id="Ul1"> -- this ID is unique for each of the tabs

#need the website associated with each of the items, as well as the daa
'''
From the below we need: 1. href (Transcript url), and item_pub date
<a class="item-title" href="/media/releases/2022/p0331-youth-mental-health-covid-19.html">New CDC data illuminate youth mental health threats during the COVID-19 pandemic</a> -
 <span class="item-pubdate">Thursday, March 31, 2022</span>
</li>
'''


'\nFrom the below we need: 1. href (Transcript url), and item_pub date\n<a class="item-title" href="/media/releases/2022/p0331-youth-mental-health-covid-19.html">New CDC data illuminate youth mental health threats during the COVID-19 pandemic</a> -\n <span class="item-pubdate">Thursday, March 31, 2022</span>\n</li>\n'

# Get the Text of the Transcript

In [79]:


'''
Include the following items:
"Media Telebriefing"
"Transcript"
"Statement"


'''
num_transcripts = 0 
transcript_words = ["Media Telebriefing", "Transcript", "Statement"]

for pub_date in list(transcript_info.keys()):
      # print(pub_date)
       #get needed information
       transcript_url = transcript_info[pub_date]["transcript_url"]
       transcript_title = transcript_info[pub_date]["transcript_title"]

       if any(word in transcript_title for word in transcript_words): #only include the transcripts
              num_transcripts += 1

       #        #go to the url
       #        show_page = requests.get(transcript_url)

       #        #convert the page into a soup object
       #        show_soup =  BeautifulSoup(show_page.content, "html.parser")

  
       #        #get the content of the transcript = <div class= "card-body bg-white">
       #        transcript_text = show_soup.find(class_="card-body bg-white").text

       #        #add the transcript text to the dictionary
       #        transcript_info[pub_date]["transcript_text"] = transcript_text
       #        #print(transcript_title, transcript_url, transcript_text)

       # list_pub_date = pub_date.split()
       # print(list_pub_date)
             



In [80]:
#How many transcripts are there?

print("There are {} transcripts from late 2019 to 2022 from the CDC on COVID-19.".format(num_transcripts))


There are 48 transcripts from late 2019 to 2022 from the CDC on COVID-19.


In [85]:
#create a function 

def get_transcript(folder_name):
    '''
    Input: folder_name (string)  (no need for search URL here because it does change)
    Output: None (does not return anything but writes the text of the transript to a file)

    '''

    # ----------  GET TO THE WEBISTE & EXTRACT HTML
    #store the cdc url.
    cdc_newsroom_url = "https://www.cdc.gov/media/dpk/diseases-and-conditions/coronavirus/coronavirus-2020.html"

    #step 2: get to the landing page of the website
    landing_page = requests.get(cdc_newsroom_url)

    #step 3: convert page into html
    soup = BeautifulSoup(landing_page.content, "html.parser")



    #------ PARSE THROUGH THE HTML
    #step 1: gets all of the contents of each of the tabs (Advisories, Trasnricripts, etc)

    press_release_feed = soup.find_all("div", class_= "pressrel")  #this will include in

    transcript_info = {} #key = air date, value is a dictionary with the (transcript url, guest, and text of the show)
    starting_url = "https://www.cdc.gov"

    for tab in press_release_feed:

    
        item_titles = tab.find_all("a", class_="item-title")
        pub_dates = tab.find_all("span", class_="item-pubdate")
    
        for (title, date) in zip(item_titles, pub_dates):
            transcript_url = starting_url + title["href"]
            date_text = date.text
            title_text = title.text
            #print(date_text, title_text, transcript_url, "\n")

            #add the information to the transcript info to the dictionary
            transcript_info[date_text] =  {"transcript_url": transcript_url,
                                            "transcript_title": title_text }

    
    # --- GET THE ACTUAL TRANSCRIPT TEXT

    #step 7: iterate through all of the pub dates to get the actual text of the transcript + write the text to a file 

    transcript_words = ["Media Telebriefing", "Transcript", "Statement"] #only want to include transcripts
    for pub_date in list(transcript_info.keys()):
       #get needed information
       transcript_url = transcript_info[pub_date]["transcript_url"]
       transcript_title = transcript_info[pub_date]["transcript_title"]

       issue_url = "https://www.cdc.gov/media/releases/2020/t0229-COVID-19-update.html"

       if any(word in transcript_title for word in transcript_words): #only include the transcripts
           if transcript_url != issue_url:

                #go to the url
                show_page = requests.get(transcript_url)

                #convert the page into a soup object
                show_soup =  BeautifulSoup(show_page.content, "html.parser")


                #get the content of the transcript = <div class= "card-body bg-white">
                transcript_text = show_soup.find(class_="card-body bg-white").text

                #add the transcript text to the dictionary
                transcript_info[pub_date]["transcript_text"] = transcript_text
                #print(transcript_title, transcript_url, transcript_text)



        # --- NOW THAT WE HAVE THE ACTUAL TEXT OF THE TRANSCRIPT, EXPORT THIS TO A TEXT FILE
            
                #create a file name + output file path
                list_pub_date = pub_date.split()


                filename = list_pub_date[1] + "_" + list_pub_date[2][:-1] + "_" + list_pub_date[3] + "_" + "cdc" + "_" + "transcript.txt"
                repo_path = os.path.dirname(os.getcwd())
                output_path = os.path.join(repo_path, "data", "01-raw", folder_name, filename)

                #print(output_path)

                #export the file
                with open(output_path,"w") as f:
                    f.write(transcript_text)







In [86]:
#run the function in the above cell to get the CDC transcripts
folder_name = "cdc_press_releases"

get_transcript(folder_name) #running to a slight issue here!  with the below URL
# //www.cdc.gov/media/releases/2020/t0229-COVID-19-update.html

ConnectionError: HTTPSConnectionPool(host='www.cdc.govhttps', port=443): Max retries exceeded with url: //www.cdc.gov/media/releases/2020/t0229-COVID-19-update.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd346e67fd0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))